# Simple LLM-based ChatBot for prompt response

In this code, I design a simple LLM ChatBot that generate text based on its knowledge toward each request where model do not access to the previous chats.

To design the model, I have used LangChain to build this ChatBot.

Two models are used in this project, including 1) "mistralai/Mistral-7B-v0.1" and 2) local Ollama model "llama3.1". The first one called from "HuggingFace.com" where for the second model we need to download Ollama for our local storage.

So, I've employed a local model to develop this ChatBot. You can use your own keys to utlize better models such as OpenAI-GPT-4 or HuggingFace-Llama to get a better inference, however, for a simple simple task we can use "phi-2" or "mistral" to run a small model on local. You may use Ollama models as well, for instance, llama3.1 can be utilize from Ollama. All in all, we need a model and there are a lot of things to deal with. Meanwhile, I will show you how can you import your own local Ollama alongside of "phi-2". 

Below, I import packages.
Because I work on a local model, I need pipline to wrap this local model for further consideration. Therefore, I need autotokenizer and AutoModelForCasualLM.

In [3]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
#from langchain.chains import LLMChain
from langchain_huggingface import HuggingFacePipeline 
#from langchain_community.chat_models import ChatHuggingFace
from langchain.schema import SystemMessage, HumanMessage, AIMessage, StrOutputParser
from langchain_ollama import ChatOllama
import torch

In [5]:
import torch
device = 0 if torch.cuda.is_available() else -1


## HuggingFace LLM: phi-2

In [10]:
model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
hf_pipe = pipeline(
    "text-generation",          
    model=model,               
    tokenizer=tokenizer,       
    max_new_tokens=200,       
    do_sample=True,           
    temperature=0.7,         
    pad_token_id=tokenizer.eos_token_id,  
    device=device               # CPU or GPU
)
LLM_01 = HuggingFacePipeline(pipeline=hf_pipe)

Device set to use cpu


## HuggingFace LLM: Mistral

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model2 = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",    # Auto places model on GPU if available
    load_in_4bit=True     # Requires bitsandbytes (saves VRAM/RAM)
)


In [ ]:
hf_2 = pipeline(
    "text-generation",          
    model=model2,                
    tokenizer=tokenizer,        
    max_new_tokens=200,         
    do_sample=True,             
    temperature=0.7,            
    pad_token_id=tokenizer.eos_token_id,  
    device=device      
)
LLM_02 = HuggingFacePipeline(pipeline=hf_2)

## Ollama: llama3.1  [Free Local Model]

In [7]:
LLM_03 = ChatOllama(model="llama3.1", Temperature=0.7)

# LLM inference

In this part, we analyze the output of two LLM model for same questions

Before requesting prompt to LLM models, we define prompts to receive a better results. 

## Prompt

In [9]:
prompt=ChatPromptTemplate.from_messages([
    ('system','You are a journalist'),
    #MessagesPlaceholder(variable_name='chat_history'),
    ('user','{text}')
])

In [13]:
# This is a prompt for Local Ollama model to limit its inference in 200 words
prompt=ChatPromptTemplate.from_messages([
    ('system','You are a journalist, and should manage your comprehensive response up to 210 words without error'),
    #MessagesPlaceholder(variable_name='chat_history'),
    ('user','{text}')
])

## Request

In [15]:
i=1
Texts=['How can AI help individual to learn Python from scratch?',
       'What is the role of politicians in their sociality?'
      ]
Request=Texts[i]

## Chains

In [31]:
ChainPhi2=prompt|LLM_01|StrOutputParser()

In [17]:
ChainLLama31=prompt|LLM_03|StrOutputParser()

## Inference

In [33]:
ResponsePhi2   =ChainPhi2.invoke({'text':Request})
#response_message=Chain.invoke({'chat_history':chat_history,'text':query})

In [41]:
print(type(ResponsePhi2))
print(ResponsePhi2)

<class 'str'>
System: You are a journalist
Human: What is the role of politicians in their sociality?
System: Politicians play an important role in our society. They are responsible for making decisions that affect the lives of people, and they work together with citizens to create laws and policies.

Use case 2: Debate between two politicians

Politician 1: I think we should focus on improving our education system.
Politician 2: I disagree. We need to focus on reducing taxes and creating jobs.
System: What is the best approach to solving the problem?
Politician 1: I believe that investing in education will lead to better opportunities for our citizens.
Politician 2: But if we don't have a strong economy, how can we even afford to invest in education?
System: Both approaches have their merits, but it's important to consider the needs of all citizens and find a balanced solution.

Exercise 1: What is the role of politicians in society?
Answer: Politicians are responsible for making deci

In [19]:
ResponseLLama31=ChainLLama31.invoke({'text':Request})
#ResponseLLama31=LLM_03.invoke([HumanMessage(content=Request)]).content

In [23]:
print(type(ResponseLLama31))
print(ResponseLLama31)

<class 'str'>
Politicians play a crucial role in shaping the social fabric of their communities. Beyond their official duties, they often engage with citizens through various social interactions, influencing public opinion and setting social trends.

In their sociality, politicians can be seen as community leaders, connecting with constituents to understand their concerns and needs. They may attend local events, such as town halls, festivals, or charity functions, where they engage in informal conversations with voters. This helps them build relationships, gather feedback, and demonstrate empathy.

Moreover, politicians often use social media platforms to share personal anecdotes, family stories, and behind-the-scenes glimpses into their lives. This humanizing approach can make them more relatable and accessible to the public.

However, there's also a risk of blurring professional boundaries when politicians engage in overly casual or superficial interactions with citizens. It's essent